# Natural Language Processing

## Tools for NLP

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

### Tokenizers

In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [0]:
sen_df = spark.createDataFrame([
    (0,'Hi I heard about Spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')
],['id','sentence'])
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [0]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='words',pattern='\\W')

In [0]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [0]:
tokenized = tokenizer.transform(sen_df)
tokenized.withColumn('tokens',count_tokens(tokenized['words'])).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [0]:
rg_tokenized = regex_tokenizer.transform(sen_df)
rg_tokenized.withColumn('tokens',count_tokens(rg_tokenized['words'])).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



### Stop words remover

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
sentenceDataFrame = spark.createDataFrame([
    (0, ['I','saw','the','green','horse']),
    (1, ['Mary','had','a','little','lamb'])
], ['id','tokens'])
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [0]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



### N-gram

In [0]:
from pyspark.ml.feature import NGram

In [0]:
wordDataFrame = spark.createDataFrame([
    (0, ['Hi','I','heard','about','Spark']),
    (1, ['I','wish','Java','could','use','case','classes']),
    (2, ['Logistic','regression','models','are','neat'])
],['id','words'])

In [0]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



### TF-IDF
Term Frequency and Invert Document Frequency

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [0]:
sentenceData = spark.createDataFrame([
    (0.0,'Hi I heard about Spark'),
    (0.0, 'I wish java could use case classes'),
    (1.0, 'Logistic regression models are neat')
],['label','sentence'])
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [0]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')
words_data = tokenizer.transform(sentenceData)

In [0]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')
featurized_data = hashing_tf.transform(words_data)
featurized_data.select('rawFeatures').show(truncate=False)

+------------------------------------------------------------------------------------+
|rawFeatures                                                                         |
+------------------------------------------------------------------------------------+
|(262144,[18700,19036,33808,66273,173558],[1.0,1.0,1.0,1.0,1.0])                     |
|(262144,[19036,20719,55551,58672,98717,109547,192310],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(262144,[46243,58267,91006,160975,190884],[1.0,1.0,1.0,1.0,1.0])                    |
+------------------------------------------------------------------------------------+



In [0]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = idf.fit(featurized_data)
rescaled_data = idf_model.transform(featurized_data)
rescaled_data.select('label','features').show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

### Count Vectorizer

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
    (0, 'a b'.split(" ")),
    (1, 'a b b c a'.split(" "))
], ['id', 'words'])

In [0]:
cv = CountVectorizer(inputCol='words', outputCol='features',
                    vocabSize= 3, minDF= 2.0)
model = cv.fit(df)
results = model.transform(df)
results.show(truncate=False)

+---+---------------+-------------------+
|id |words          |features           |
+---+---------------+-------------------+
|0  |[a, b]         |(2,[0,1],[1.0,1.0])|
|1  |[a, b, b, c, a]|(2,[0,1],[2.0,2.0])|
+---+---------------+-------------------+



## Code along
Spam model detection

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlps').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/SMSSpamCollection', inferSchema=True,sep='\t')

In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import length
data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [0]:
# difference in lengths
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham| 71.4545266210897|
| spam|138.6706827309237|
+-----+-----------------+



In [0]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer)

In [0]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [0]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

In [0]:
data_train, data_test = data.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages= [to_numeric, tokenizer,stop_remove,count_vec,idf,clean_up])

In [0]:
# fit only on train to avoid data leakage
cleaner = data_prep_pipeline.fit(data_train)
clean_data = cleaner.transform(data)

In [0]:
clean_data = clean_data.select('label','features')
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(10677,[7,12,23,6...|
|  0.0|(10677,[0,27,267,...|
|  1.0|(10677,[2,10,18,2...|
|  0.0|(10677,[0,70,85,1...|
|  0.0|(10677,[36,126,27...|
|  1.0|(10677,[9,62,134,...|
|  0.0|(10677,[9,49,107,...|
|  0.0|(10677,[125,212,4...|
|  1.0|(10677,[1,48,141,...|
|  1.0|(10677,[0,1,10,31...|
|  0.0|(10677,[20,43,115...|
|  1.0|(10677,[8,16,41,8...|
|  1.0|(10677,[10,25,48,...|
|  0.0|(10677,[40,96,190...|
|  0.0|(10677,[525,1903,...|
|  1.0|(10677,[25,97,108...|
|  0.0|(10677,[76,238,27...|
|  0.0|(10677,[0,2,45,12...|
|  0.0|(10677,[0,68,105,...|
|  1.0|(10677,[4,25,33,6...|
+-----+--------------------+
only showing top 20 rows



In [0]:
training, testing = clean_data.randomSplit([0.7,0.3])

In [0]:
spam_detector = nb.fit(training)
test_results = spam_detector.transform(testing)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(10677,[0,1,2,7,8...|[-779.23874585862...|[1.0,3.6497186059...|       0.0|
|  0.0|(10677,[0,1,4,50,...|[-808.53395125265...|[1.0,1.5802959600...|       0.0|
|  0.0|(10677,[0,1,13,14...|[-526.60140181892...|[1.0,1.3670460396...|       0.0|
|  0.0|(10677,[0,1,17,21...|[-673.07200730310...|[1.0,1.1765263079...|       0.0|
|  0.0|(10677,[0,1,21,31...|[-569.66823420485...|[1.0,1.9159894818...|       0.0|
|  0.0|(10677,[0,1,25,11...|[-494.78472874627...|[1.0,2.2506856832...|       0.0|
|  0.0|(10677,[0,1,31,32...|[-656.01071264983...|[1.0,1.5046500969...|       0.0|
|  0.0|(10677,[0,1,44,17...|[-333.07211047891...|[0.99914322255448...|       0.0|
|  0.0|(10677,[0,1,401,6...|[-225.45725650895...|[0.99999999999999...|       0.0|
|  0.0|(10677,[0

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evel = MulticlassClassificationEvaluator()
acc = acc_evel.evaluate(test_results)

In [0]:
print('ACC of NB model')
print(acc)

ACC of NB model
0.935796859041093
